Training set size

In [1]:
ts = 1000

Number of modes, which will be used to generate class membership

In [2]:
nc = 5

Maximum number of iterations for the learning step

In [3]:
maxit = 1000

In [4]:
import os
import sys
import glob

sys.path.append('/opt/OTB-6.2.0/lib/python')
sys.path.append('/opt/OTB-6.2.0/lib/libfftw3.so.3')
os.environ['OTB_APPLICATION_PATH'] = '/opt/OTB-6.2.0/lib/otb/applications'
os.environ['LD_LIBRARY_PATH'] = '/opt/OTB-6.2.0/lib'
os.environ['ITK_AUTOLOAD_PATH'] = '/opt/OTB-6.2.0/lib/otb/applications'
os.environ['GDAL_DATA'] = '/opt/anaconda/share/gdal/'
import otbApplication

In [5]:
data_path = '/tmp/workspace/data'

In [6]:
s2_identifier = 'S2A_MSIL1C_20170909T060631_N0205_R134_T42SVG_20170909T061506'

In [ ]:
s2_reference = ''

In [7]:
search_expression = os.path.join(data_path, s2_identifier, s2_identifier + '.SAFE', 'manifest.safe')

In [8]:
s2_manifest = glob.glob(search_expression)[0] 

In [9]:
cwd = os.getcwd()
granule_path = os.path.join(data_path, s2_identifier, s2_identifier + '.SAFE', 'GRANULE')
os.chdir(granule_path)

In [10]:
granules = [ name for name in os.listdir('.') if os.path.isdir(name)]

os.chdir(cwd)

granules

['L1C_T42SVG_A011571_20170909T061506']

In [11]:
granule_index = 0

s2_bands = []

for band in [2, 3, 4, 8]:

    search_expression = os.path.join(granule_path, granules[granule_index], 'IMG_DATA', '*_B0' + str(band) + '.jp2')
    s2_bands.append(glob.glob(search_expression)[0])

In [12]:
OTB_app1 = otbApplication.Registry.CreateApplication('ConcatenateImages')

OTB_app1.SetParameterStringList('il', s2_bands)
OTB_app1.SetParameterString('out', 'concat.tif')

OTB_app1.ExecuteAndWriteOutput()

0

In [13]:
OTB_app2 = otbApplication.Registry.CreateApplication('KMeansClassification')

OTB_app2.SetParameterString('in', 'concat.tif')
OTB_app2.SetParameterInt('ts', ts)
OTB_app2.SetParameterInt('nc', nc)
OTB_app2.SetParameterInt('maxit', maxit)
OTB_app2.SetParameterString('out', 'classification.tif')

OTB_app2.ExecuteAndWriteOutput()

0

In [14]:
os.remove('concat.tif')

In [15]:
file = open('classification.lut', 'w') 
 
file.write('0 51 153 255') 
file.write('1 255 255 204') 
file.write('2 229 255 204') 
file.write('3 204 255 255') 
file.write('4 255 204 255') 
 
file.close() 

In [18]:
OTB_app3 = otbApplication.Registry.CreateApplication('ColorMapping')

OTB_app3.SetParameterString('in', 'classification.tif')
OTB_app3.SetParameterString('method', 'custom')
OTB_app3.SetParameterString('method.custom.lut', 'classification.lut')
OTB_app3.SetParameterString('out', 'preview.tif')
OTB_app3.SetParameterOutputImagePixelType('out', otbApplication.ImagePixelType_uint8)

OTB_app3.ExecuteAndWriteOutput()

0

In [20]:
os.remove('classification.lut')